In [1]:
import os
os.chdir("../")

In [ ]:
# from https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python

In [8]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.image as mpimg
from data.affwild2_dataset import AffWild2ExprDataset
from torchvision import transforms
from tqdm.notebook import tqdm

In [3]:
mean = [0.4722, 0.3625, 0.3350]
std = [0.2181, 0.1921, 0.1855]

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])
skip = 0
dataset = AffWild2ExprDataset(train=False, skip=skip, remove_mismatch=True,
                                    compress=True, transform=transform_valid, binary=True)

In [4]:
sample, label = dataset[10000]

/kuacc/users/nnayal17/.conda/envs/nazir_env/lib/python3.8/site-packages/torchvision/transforms/functional.py:74: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [5]:
sample.shape, label

(torch.Size([3, 112, 112]), 0)

In [9]:
data_len = len(dataset)

In [11]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tflite_models/resnet18.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

correct = 0
for i in tqdm(range(data_len)):
    sample, label = dataset[i]
    
    img = sample
    img = img[None,:,:,:]
    input_data = np.array(img, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    pred = np.argmax(output_data[0])
    if pred == label:
        correct += 1

In [58]:
print(f"Inference Test Accuracy {100 * correct / len(dataset)}")

Inference Test Accuracy 60.7559282064719


In [60]:
print(f"Inference Test Accuracy {100 * correct / len(dataset)}")

Inference Test Accuracy 66.23870037992926


In [23]:
np.argmax([1, 2])

1

In [22]:

# Test the model on random input data.
input_shape = input_details[0]['shape']
#img = mpimg.imread('samples/test.jpg').transpose(2, 0, 1)
img = sample.transpose(2, 0, 1)
img = img[None,:,:,:]
input_data = np.array(img, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[-0.04790531  0.41730937]]


In [ ]:
cp /datasets/affwild2_latest/cropped_aligned_images/144/07804.jpg /scratch/users/nnayal17/moody_much/MoodyMuch/ml/samples/

In [4]:
!cp /datasets/affwild2_latest/cropped_aligned_images/186/01265.jpg /scratch/users/nnayal17/moody_much/MoodyMuch/ml/samples/

In [54]:
import torch
import torch.nn as nn
from models.mobilenetv2 import mobilenetv2
from torchvision.models import resnet18
from tqdm.notebook import tqdm

In [33]:
resnet_model = resnet18(num_classes=2)
resnet_model.load_state_dict(torch.load("chosen_models/resnet18.pth"))

<All keys matched successfully>

In [39]:
out

tensor([[-0.2307,  0.1517]], grad_fn=<AddmmBackward>)

In [42]:
(out[0])

tensor([-0.2307,  0.1517], grad_fn=<SelectBackward>)

In [43]:
torch.argmax(out[0]).item()

1

In [49]:
c = 0
resnet_model.eval()
for i in range(data_len):
    sample, label = dataset[i]
    
    out = resnet_model(torch.from_numpy(sample.astype(np.float32)).unsqueeze(0).permute(0, 3, 1, 2))
    pred = torch.argmax(out[0]).item()
    if pred == label:
        c += 1

In [50]:
print(f"Original Model Accuracy: {100 * c / data_len}")

Original Model Accuracy: 75.67797720424473


In [51]:
mobilenetv2_model = mobilenetv2()
mobilenetv2_model.classifier = nn.Linear(mobilenetv2_model.classifier.in_features, 2)
mobilenetv2_model.load_state_dict(torch.load("chosen_models/mobilenetv2.pth"))

<All keys matched successfully>

In [55]:
c = 0
mobilenetv2_model.eval()
for i in tqdm(range(data_len)):
    sample, label = dataset[i]
    
    out = mobilenetv2_model(torch.from_numpy(sample.astype(np.float32)).unsqueeze(0).permute(0, 3, 1, 2))
    pred = torch.argmax(out[0]).item()
    if pred == label:
        c += 1

In [56]:
print(f"Original Model Accuracy: {100 * c / data_len}")

Original Model Accuracy: 76.47058823529412
